In [2]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

from shapely.geometry import Point, LineString

In [60]:
scenario_var = "_scenario1"
cost_attr = 'cost'
res_folder = r"C:\Work\Code\GOST_PublicGoods\Implementations\asia_railways"
G_pickle  = os.path.join(res_folder, "cleaned_network.pickle")

#Output files
complete_volume_map = os.path.join(res_folder, "complete_equal_volume_map.csv")
trade_volume_map = os.path.join(res_folder, "trade_volume_map.csv")

#Read in and process input data
edges_geography = r"S:/REGIONAL/ECA/Projects/Rail_Network_Mapping/data/edges.shp"
edges_data = gpd.read_file(edges_geography)

nodes_geography = r"S:/REGIONAL/ECA/Projects/Rail_Network_Mapping/data/rail_nodes.csv"
nodes_data = pd.read_csv(nodes_geography)
nodes_data['Point'] = [Point(x, y) for x, y in zip(nodes_data['X'],nodes_data['Y'])]
nodes_data['node_id'] = nodes_data['node_id'].astype(str)

OD_df = pd.read_csv(os.path.join(res_folder, "asia_railways_output_OD_matrix{}.csv".format(scenario_var)), index_col=0)
G = nx.read_gpickle(G_pickle)

trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))

# Map network volume based on ALL least-cost trips

In [19]:
def tabulate_edges(route):
    edge_table = []
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
    return(edge_table)

def get_route(route, nodes_data):
    all_nodes = []
    for x in route:
        try:
            cur_node = nodes_data.loc[nodes_data['node_id'] == x]
            all_nodes.append(Point(cur_node.X, cur_node.Y))
        except:
            pass
    if len(all_nodes) > 1:
        return(LineString(all_nodes))
    else:
        return(None)

In [34]:
all_res = []
complete_edges = []
for origin, row in OD_df.iterrows():    
    for destination, value in row.items():
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        route = nx.bidirectional_shortest_path(G, origin, destination)
        edge_table = tabulate_edges(route)
        complete_edges = complete_edges + edge_table 
        #all_res.append([origin, destination, get_route(route, nodes_data)])


In [37]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd'])
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]
complete_count.to_csv(complete_volume_map)

# Map specific routes with trade flows

In [55]:
trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))
complete_edges = []
for idx, row in trade_flows.iterrows():    
    origin = row['Node number']
    destination = row['Node Number']
    route = nx.bidirectional_shortest_path(G, origin, destination)
    edge_table = tabulate_edges(route)
    [x.append(int(row['Trade volume'].replace(",", ""))) for x in edge_table]
    complete_edges = complete_edges + edge_table 


In [61]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd', 'w'])
complete_count = pd.DataFrame(complete.groupby(['o','d']).sum())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]
complete_count.to_csv(trade_volume_map)